In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from __future__ import division
from scipy.io import loadmat
from collections import defaultdict
from IPython.core.debugger import set_trace


In [2]:
digits = loadmat('../data/digits.mat')
X_train = digits['X']
y_train = np.squeeze(digits['y'])
y_train = np.where(y_train == 10, 0, y_train)

In [3]:
X_train.shape

(5000, 400)

In [4]:
y_train.shape

(5000,)

In [5]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [6]:
n_samples = X_train.shape[0]
n_features = X_train.shape[1]
n_labels = len(np.unique(y_train))
labels = np.unique(y_train)

In [7]:
X = tf.placeholder(name='X', dtype=tf.float32, shape=(None, n_features))
Y = tf.placeholder(name='Y', dtype=tf.float32, shape=(None, 1))

thetas = tf.Variable(tf.zeros(shape=(n_features, 1), dtype=tf.float32), name='thetas')
theta0 = tf.Variable(0.0, dtype=tf.float32, name='theta0')

z = tf.matmul(X, thetas) + theta0
sigmoid = 1.0 / (1.0 + tf.exp(-z))

cost_function = tf.reduce_mean(-((Y * tf.log(sigmoid)) + (1 - Y) * tf.log(1 - sigmoid)))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.03).minimize(cost_function)

classifiers = np.zeros(shape=(n_labels, n_features+1))
class_probabilities = np.zeros(shape=(n_samples, n_labels))

with tf.Session() as sess:
    for i, label in enumerate(labels):
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {
            X : X_train,
            Y : (y_train.reshape(-1, 1) == label).astype(int)
        }
        
        for step in range(5000):
            sess.run(optimizer, feed_dict)
            
        tt0 = sess.run(theta0)
        tts = sess.run(thetas)
        
        classifiers[i, :] = [tt0] + list(tts)
        class_probabilities[:, i] = sess.run(sigmoid, { X: X_train }).T

In [15]:
y_pred = np.argmax(class_probabilities, axis=1)

In [16]:
accuracy = sum(y_pred == y_train) / n_samples

In [17]:
accuracy

0.906